# Parsing

Автор ноутбука - Дуркин Анатолий Альбертович

Преподаватель кафедры прикладной математики и компьютерных наук СГУ им. Питирима Сорокина

Замечания, предложения, идеи, вопросы, связь с автором:
- anatoliy.durkin@mail.ru
- Telegram - @AnatoDu

Больше информации и материалов на канале автора: https://t.me/smth_on_it

## Пара слов о парсинге

Зачастую данные, которые нам нужны для анализа, можно найти в готовом и удобном виде в открытых банках данных или в иных местах. Но иногда данные присутствуют только в определённом месте, да ещё и не в самой удобной для анализа форме. Тут нам может пригодиться парсинг.

Парсинг данных — это извлечение структурированной информации из неструктурированных или полуструктурированных данных. Проще говоря, парсинг — это способ «прочитать» данные и преобразовать их в формат, который удобен для анализа и использования.

Парсить можно разные форматы: текстовые файлы, электронные таблицы, XML, JSON и многое другое.

Здесь и сейчас мы рассмотрим парсинг веб-сайта. А для примера возьмём сайт https://167000.ru

## Получаем данные с сайта

Если у вас есть сайт, и вам необходимо информацию с него привести в удобный для анализа вид, какие шаги вы бы предприняли?

1. Получить информацию с сайта. В любом виде, в каком это возможно. Мы знаем, что страница сайта может быть представлена в виде HTML-кода, вот в таком виде мы её и постараемся получить, а поможет нам в этом библиотека `requests`.
2. Найти требуемые данные. Полученный на предыдущем шаге материал явно содержит много лишнего, но где-то в нём есть и то, что мы хотим получить. Однако, как это найти? Как мы уже сказали, получили мы формат HTML, а у него есть свои строгие правила построения. Разобрать полученную страницу на теги для дальнейшего поиска нужной информации поможет библиотека `BeautifulSoup`.
3. Собрать данные. Когда поиск выполнить легко, можно добраться до нужной информации и поместить её в более удобьный вид, например, во фрейм данных. Здесь используем библиотеку `pandas`.

Итак, план определён, инструменты упомянуты. Начнём работу. Подключим требуемые библиотеки.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Кстати, если у вас не установлена какая-то библиотека, её легко доустановить прямо из Jupyter-ноутбука. Консольную команду можно выполнить, просто добавив восклицательный знак.

In [ ]:
# Этот код установит библиотеку (по крайней мере при работе в локальной версии):
# !pip install bs4

Теперь получим страницу сайта с помощью функции `get`. Мы будем брать не соновную страницу, так как на ней мало полезного, а зайдём в раздел "Продажа".

In [ ]:
page = requests.get('https://167000.ru/syktyvkar-i-prigorody/prodam/')

In [ ]:
page

Как результат выполнения функции мы получили в переменной лишь строку `<Response [200]>`, что явно не похоже на страницу в интернете. Однако, именно такой результат говорит нам о том, что всё хорошо (если вы получили не "200", у вас какие-то проблемы с доступом к сайту). Код "200" означает, что код страницы успешно получен и хранится в переменной. Получить его очень просто:

In [ ]:
src = page.text

In [ ]:
src

Что ж, это и правда похоже на HTML, только сваленный в кучу. Где-то можно увидеть теги, их аттрибуты, но искать тут что-то конкретное невозможно.

Здесь нам на помощь приходит следующая библотека, она может разобрать этот код по тегам и представить его в более приятном виде.

In [ ]:
soup = BeautifulSoup(src, 'lxml')

В данной функции вторым аргументом принимается название парсера, который будет использоваться для "разбирания" текста. Функция умеет разбирать многие форматы, поэтому такое указание важно. Здесь мы применяем `lxml`, так как это хороший парсер для html. Он может быть не установлен, но вы всегда можете доустановить его, либо использовать аналог - `html.parser`, либо не указывать ничего, так как по умолчанию установлен как раз второй вариант.

Посмотрим, что получилось.

In [ ]:
soup

## Ищем теги

Текст стал значительно более удобочитаем. Более того, функция не просто разбирает текст по тегам и оформляет это красово, она ещё и подготавливает его к возможности удобного поиска по коду.

Чтобы понимать, что мы ищем, в браузере на старнице сайта жмём правую кнопку и выбираем "Просмотреть код" (так в Google Chrome). Нам нужно, чтобы у нас вместе были и страница, и её код. Теперь в открывшемся разделе с кодом страницы следует найти кнопку, позволяющую на сайте выбрать объект. Обычно он выглядит как стрелочка с квадратиком или без. Если вы нашли эту кнопку, то теперь при наведении на объект на странице сайта вы увидите, где в коде описан этот элемент.

Таблица с предложениями о продаже, а также фильтры и прочие мелочи лежат в одном теге: `<main class="main-content">`. Чтобы найти именно этот тег, воспользуемся методом `find`. Он возвращает первый встретившийся тег, удовлетворяющий условиям, и возвращает его вместе со всем его содержимым.

Сразу отметим, что одинаковых тегов может быть много, а метод вернет самый первый подходящий. Чтобы уточнить параметры поиска, можно указать атрибуты тега. Так, `class` можно указать сразу же, просто записав его через запятую к тегу.

Итак, найдем тег.

In [ ]:
soup.find('main', 'main-content')

Мы действительно получили искомый тег с его содержимым. Очень удобно, а главное, быстро и легко.

## Собираем таблицу

### Начинаем с заголовков

Давайте рассмотрим таблицу подробнее. Нам необходимо получить из неё все данные, а также, желательно, заголовки, чтобы не печатать их самостоятельно (да, их мало, и напечатать их легко, но мы же учимся парсить). Можно отметить, что заголовки лежат в теге `<div id="article">` в одной таблице, а все остальные строки в отдельной таблице. Попробуем достать заголовки.

Поиск можно осуществить тем же самым методом, но с одним важным замечанием: в данном случае у нас указан атрибут `id`, а не `class`, поэтому немного меняется способ его указания в методе. В данном случае мы должны составить словарь, где ключом является имя атрибута, а значением - значение данного атрибута.

Поскольку предыдущий результат мы не сохранили, повторяем его, а затем ищем следующий тег.

In [ ]:
main_content = soup.find('main', 'main-content')
div = main_content.find('div', {'id':'article'})

Маленькая вставка про такое правило как "Бритва Оккама". Это методологический принцип, в кратком виде гласящий: "Не следует множить сущности без необходимости".

Посмотрите на предыдущую ячейку, мы сохраняет результат поиска в `main_content`, а затем ищем внутри этой переменной и записываем результат в переменную `div`. Но `main_content` нам больше не понадобится в дальнейшей работе, а значит, эту переменную можно было не создавать. Она лишь занимает место и имя, в дальнейшем разбираться с такими "лишними" переменными может быть сложно.

Старайтесь помнить о "Бритве Оккама" и применять её по возможности. Применим её тут и запишем наш двойной поиск в сокращённом виде:

In [ ]:
div = soup.find('main', 'main-content').find('div', {'id':'article'})

Не обязательно всегда использовать для поиска тега `find`. Например, сейчас, если посмотрим на код, то мы видим, что в нашем `div` на следующем уровне вложенности лежат все нужные нам таблицы - и с заголовками, и с предложениями. Причём таблица с заголовками первая. Поэтому к ней мы можем обратиться через точку. Она сработает так же, как `find`, но если послений ищет на всех уровнях вложенности, то через точку мы попадаем только на следующий уровень.

А в нашей таблице лежит тег `<tr>`, задающий строку с нужными нам заголовками, поэтому обратимся сразу и к нему:

In [ ]:
div.table.tr

Теперь по элементам внутри найденного нами тега можно просто пройти циклом. А чтобы получить то, что находится между открывающим и закрывающим тегом, обратимся к атрибуту `text`. Соберём все заголовку в один список.

В последней строке мы хитрим: при проходе циклом по содержиому тега мы получаем пустые элементы каждый второй раз, от которых нужно избавиться.

In [ ]:
columns = []
for th in div.table.tr:
    columns.append(th.text)
columns = columns[1::2]

In [ ]:
columns

Все заголовки получены, можно подготовить пустой DataFrame для его дальнейшего заполнения.

In [ ]:
df = pd.DataFrame(columns=columns)

In [ ]:
df

### Парсим все данные

Место для сохранения данных готово, теперь надо подумать, что будет индексами таблицы. Конечно, можно просто нумеровать с единицы и по порядку, но мы не ищем лёгких путей. И, к тому же, если мы захотим продолжить заполнять таблицу и будем парсить периодически, раз в неделю, например, как отслеживать, что такое предложение в нашей таблице уже есть?

Обратимся к странице. Каждая строка с предложением о продаже задаётся тегом `<a>`, у которого есть атрибут `href`, в котором, в свою очередь, находится ссылка на страницу с этим предложением. Очевидно, что в этой ссылке лежит уникальное id-значение для этого предложения. Именно этот id и возьмём за индексы.

Для начала посмотрим, сколько у нас таких тегов. Теперь мы воспользуемся методом `find_all`, он находит не только первый встретившийся, а все указанные теги.

In [ ]:
len(div.find_all('a'))

Судя по странице, их должно быть меньше. Посмотрим, сколько у нас таблиц. Не забываем, что первая таблица хранит заголовки, поэтому её исключаем.

In [ ]:
len(div.find_all('table')[1:])

Теперь их нужное количество. Но как же тогда отобрать только нужные нам теги `a`? Если посмотреть на сайт и код, то видно, что "лишние" теги лежат на следующем уровне вложенности, а на первом их как раз нужное нам количество. Чтобы ограничить поиск по другим уровням, можно указать аргумент `recursive=False`.

А как же получить значение атрибута? Мы научились переходить через точку к вложенному тегу, научились получать содержимое через атрибут `text`. А чтобы получить значение атрибута тега, мы должны обратиться к этому атрибуту через квадратные скобки:

In [ ]:
div.find('a')['href']

Остаётся только убрать лишние символы по краям  и у нас останется id предложения о продаже.

In [ ]:
for a in div.find_all('a', recursive=False):
    id = a['href'][3:-1]
    print(id)

Так же, пробегаясь по циклу, можно собрать и все внутренные данные.

Для сокращения кода я использую list comprehension (генератор списков, или быстрое создание списков) и тернарный оператор для исключения пустых элементов. Это очень удобная связка методов для формирования списков.

In [ ]:
for table in div.find_all('table'):
    data = [td.text for td in table.tr if td.text != '\n']
    print(data)

Всё хорошо, мы можем дважды пройти по циклу, один раз собрать в список все id, второй раз собрать в другой список все данные, а затем заполнить DataFrame. Но мы что? Правильно, лёгких путей не ищем. Поэтому попробуем объединить эти два цикла в один. Проблема в том, что если на сайте при просмотре кода тег `table` лежит внутри тега `a`, то при парсинге почему-то они оказываются последовательными. То есть мы не можем пройтись циклом по одному тегу и обращаться внутрь к другому (если у вас спарсилось вложенно, вам повезло, дерзайте!).

### Собираем таблицу

Итак, у нас есть два цикла с разными элементами. Их количество одинаковое и они попарно соответствуют друг другу, представляя одно и то же предложение о продаже. Для таких случаев прекрасно подходит функция `zip`, она позволяет попарно собрать элементы из разных списков. Посмотрите, как она работает:

In [ ]:
a = ['a1', 'a2', 'a3', 'a4']
t = ['t1', 't2', 't3', 't4']

zip(a, t)

Ах да, простите, `zip` - итератор, и работает он "лениво", то есть, пока не попросишь, результат он не вернёт. Это зачастую удобно, чтобы не занимать зря время и память. Поэтому нужно указать, что мы хотим преобразовать итератор, например, в список.

In [ ]:
list(zip(a, t))

Как видите, мы получаем несколько кортежей, где первый элемент взят из первого списка, а второй - из второго. Удобная штука!

Итак, воспользуемся всеми последними наработками и соберём цикл. И сразу же будем записывать данные в созданный ранее DataFrame с использованием метода `loc`.

In [ ]:
for a, table in zip(div.find_all('a', recursive=False), div.find_all('table')[1:]):
    id = a['href'][3:-1]
    data = [td.text for td in table.tr if td.text != '\n']
    df.loc[id] = data

In [ ]:
df

Отлично! Собрали таблицу. Но есть небольшие странности. Они могут не отображаться в таблице (так, у меня локально не видны, но в Google Colab были видны сразу), но если посмотрим на значения отдельно, то сразу увидим:

In [ ]:
df['Адрес'].iloc[0]

In [ ]:
df['Цена'].iloc[0]

В адресах присутствует какое-то неимоверное количество пробелов, а в ценах вообще непонятные символы. Будем разбираться.

Для избавления от лишних пробелов используем замечательную связку методов `split` и `join`. Первый разделяет строку по указанному символу (или подстроке), а второй соединяет элементы списка с указанным разделителем. При этом если `split` при разделении по пробелам встретит несколько пробелов, он посчитает их за один. Воспользуемся данной связкой:

In [ ]:
text = 'Кирпичная 21                Строитель, Сыктывкар\n'

In [ ]:
' '.join(text.split())

Прекрасно!

А что с ценами? На самом деле в ценах стоит "\xa0" - код неразрывного пробела Unicod. Они нам тут не нужны для анализа, поэтому можем просто заменить их:

In [ ]:
text = '17\xa0500\xa0000'

In [ ]:
text.replace('\xa0', '')

Обе проблемы решили, их можно сразу вставить в цикл, тогда он примет такой вид:

In [ ]:
for a, table in zip(div.find_all('a', recursive=False), div.find_all('table')[1:]):
    id = a['href'][3:-1]
    data = [td.text for td in table.tr if td.text != '\n']
    data[1] = ' '.join(data[1].split())
    data[2] = data[2].replace('\xa0', '')
    df.loc[id] = data

In [ ]:
df

Великолепно!

Вдруг вспомнилась недавно изученная нами "Бритва Оккама". Конечно, никакие особо лишние сущности мы тут не множим, но можно ли упростить наш цикл?

Да, мы определённо можем не создавать переменную для id. А можем ли мы опустить использование data? А как в таком случае вносить исправления? Что ж, исправления можно внести сразу в готовый DataFrame, это будет даже удобнее и, возможно, быстрее.

Я предлагаю упростить код таким образом:

In [ ]:
df2 = pd.DataFrame(columns=columns)
for a, table in zip(div.find_all('a', recursive=False), div.find_all('table')[1:]):
  df2.loc[a['href'][3:-1]] = [td.text for td in table.tr if td.text != '\n']
df2['Адрес'] = df2['Адрес'].apply(lambda x: ' '.join(x.split()))
df2['Цена'] = df2['Цена'].str.replace('\xa0', '')

In [ ]:
df2

Мы действительно получили то же самое.

Но что за методы для внесения исправлений? DataFrame может сам пробегаться по своим строкам и вносить изменения. Это крайне удобно, когда нужно сделать одно и то же во всём столбце. Так мы поступаем с ценой, единственное отличие - добалось `.str`, это нужно, что напрямую указать, что дальше мы применяем строковый метод.

С адресами чуть сложнее. Поскольку тут нам надо применить два разных метода в связке, предыдущий способ не подойдёт. Но тут на помощь приходят метод `apply` и `lambda-функции`. Первый - это своеобразный цикл, здесь, применяемый к серии, он проходит по всем строкам и к каждой применяет указанную внутри функцию. А lambda-функции это удобнейшая вещь, когда нужно выполнить немного действий, влезающих в одну строку. Не писать же для этого полноценную функцию, а вот такой короткий вариант вполне сгодится.

## Результат

Итак, мы успешно спарсили сайт и собрали таблицу с даннными. Конечно, и в ней есть над чем поработать. Например, разделить этажи и материал дома на несколько столбцов.

Однако, это уже улучшения, которые вы можете попробовать сделать сами. А потом эту таблицу можно проанализировать.

Или нет?

## Продолжаем парсить

В таблице слишком мало данных! Для анализа не очень годится, ведь на сайте их значительно больше.

Давайте попробуем достать их оттуда. Но сделаете это вы самостоятельно, я только подскажу.

Если вы перейдёте на вторую страницу на сайте, то увидите, что в адресе появится указание страницы. Это мы и будем использовать. Зная, сколько у нас всего страниц, мы можем просто пробежаться циклом:

In [ ]:
for i in range(1, 21):
    print(requests.get(
        'https://167000.ru/syktyvkar-i-prigorody/prodam/?page='+str(i)
    ))

А ещё лучше, если цикл можно будет остановить автоматически, вдруг количество страниц поменяется, не будем же мы каждый раз смотреть. Придумайте, как можно остановить цикл.

Но не будем останавливаться на этом! Ведь в этой таблице мы видели лишь краткое описание предложения о продаже. У каждого из них есть страница, куда можно зайти и увидеть значительно больше информации. Именно это я и предлагаю сделать - спарсить информацию с этих страниц и собрать более объёмный по количеству столбцов DataFrame.

Помните, где мы взяли id? В ссылках, тех самых, нужных нам сейчас ссылках. Точнее, там были лишь вторые половинки этих ссылок, а первые - сам сайт. Получить данные с этих страниц теперь легко:

In [ ]:
for a in div.find_all('a', recursive=False):
  print(requests.get('https://167000.ru'+a['href']))

Теперь мы можем ходить по страницам, увеличивая и длину, и ширину нашей таблицы.

Попробуйте теперь собрать большую таблицу (возможно, лучше перейти на сбор информации только о квартирах https://167000.ru/syktyvkar-i-prigorody/prodam/kvartira/), а затем проанализировать её. Уверен, у вас всё получится!

In [ ]:
# ваш код